In [2]:
import os
from IPython.display import Markdown, display
from openai import OpenAI
import openai
import json

In [3]:
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter key: ")

Enter key: ··········


In [7]:
def bio_agent(user_input,entered_details,model='gpt-4.1-mini'):
  available_info = json.dumps(entered_details)
  system_prompt= f"""
You extract personal bio information ONLY from the current {user_input}.
The fields you must fill are:

1. Name
2. Date_of_birth
3. Email
4. Phone
5. Address

Output strictly in JSON only in this format:

{{
  "Name": ...,
  "Date_of_birth": ...,
  "Email": ...,
  "Phone": ...,
  "Address": ...,
  "Next Question": <the question you ask the user to get the missing input>
}}

In case if any of the information is not avaliable then just fill it with "NA".


here are the available information {available_info} you have gathered so far,Identify which which fields are not available(NA).
your next question should be based on this field.

Rules:
1. If the user_input contains ANY of the missing fields, you MUST extract it.
2. NEVER override fields that are already filled (not NA).
3. NEVER mark a field as NA if it was already filled earlier.
4. Only fields missing (NA) in available_info should be considered.
5. Ask ONLY for the next missing field.
6.If the user input contains ONLY a number or looks like a phone number (10 digits), treat it as the Phone value
7.If the user input looks like a location, area, city name, pincode, house number, or anything resembling an address, treat it as the Address field.
8.IF every details is provided by the user then break the loop



                """
  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_input}
  ]


  #3. Call OpenAI
  response = openai.chat.completions.create(model=model, messages=messages, temperature=0.0)

  #4. return the results
  content = response.choices[0].message.content




  #5. Parse JSON safely
  try:
      data = json.loads(content)
  except:
      data = {}

  return data


In [5]:
bio_data = {
    "Name": "NA",
    "Date_of_birth": "NA",
    "Email": "NA",
    "Phone": "NA",
    "Address": "NA"
}

user_input = input("Please tell me your full name:\n")

for i in range(5):
    extract = bio_agent(user_input, bio_data).0

    if bio_data["Name"] == "NA":
      bio_data["Name"] = extract["Name"]

    if bio_data["Date_of_birth"] == "NA":
      bio_data["Date_of_birth"] = extract["Date_of_birth"]

    if bio_data["Email"] == "NA":
      bio_data["Email"] = extract["Email"]

    if bio_data["Phone"] == "NA":
      bio_data["Phone"] = extract["Phone"]

    if bio_data["Address"] == "NA":
      bio_data["Address"] = extract["Address"]

    user_input = input(extract["Next Question"] + "\n")

    if (bio_data["Name"] != "NA" and
        bio_data["Date_of_birth"] != "NA" and
        bio_data["Email"] != "NA" and
        bio_data["Phone"] != "NA" and
        bio_data["Address"] != "NA"):
      break

    if user_input.lower() == "quit":
      break


Please tell me your full name:
ANIKA ADVIKA GOWDA
Could you please provide your date of birth?
JAN 01 2003
Please provide your email address.
anikagowda@gmail.com
Please provide your phone number.
9060405020
Could you please provide your address?
Bangalore 566701
NA
sorry i didt undesrtand


In [6]:
extract

{'Name': 'ANIKA ADVIKA GOWDA',
 'Date_of_birth': 'JAN 01 2003',
 'Email': 'anikagowda@gmail.com',
 'Phone': '9060405020',
 'Address': 'Bangalore 566701',
 'Next Question': 'NA'}